In [1]:
import pandas as pd

In [ ]:
!pip install pyreadstat

In [ ]:
df = pd.read_spss('ZA5597_v3-0-0.sav')
df.tail()

In [ ]:
len(list(df.columns))
len(df)
df.head()
qs = [ col for col in df.columns if col.startswith('d48')]
#qs

In [ ]:
#df[['tnscntry', 'country']].drop_duplicates()
# Nationality
keys = [f'q1_{i}' for i in range(1,29)]
df[keys].replace('Mentioned', 1).replace('Not mentioned', 0).sum(axis=1).unique()

In [ ]:

q = 'd48a_'
keys = [f'{q}{i}' for i in range(1, 40)]
df_q = df[keys + ['index']]
m = pd.melt(df_q, id_vars=['index'], value_vars=keys)
fl = m[m.value == 'Mentioned'].set_index('index')['variable'].apply(lambda x: ldict[x[len(q):]]).sort_index()
fl = fl.rename('L1')

general = ['tnscntry', 'country']

df_langs = pd.concat([fl, df['d48b'].rename('L2'), df['d48c'].rename('L3'), df['d48d'].rename('L4')], axis=1) 
df_langs

In [ ]:
with open('langs.txt') as fp:
    s = fp.read()
    langs = s.split('\n')
langs.sort()
langs = langs + ['Other']
ldict = dict(zip([str(i) for i in range(1, 40)], langs))
ldict

## Normal

In [2]:
import numpy as np

def mp(df, frm, to, ls, key):
    long = melt(df, frm, to, ls, key)
    #long = long.reset_index().sort_values(by='id').drop(key, axis=1).rename({'lang': key}, axis=1)
    long = long.set_index('id').replace({'DK': np.NaN})
    long = long[~long.index.duplicated(keep='first')]
    return long

def melt(df, frm, to, ls, key):
    to = to + 1
    keys = [f'v{i}' for i in range(frm, to)]
    df_l = df[keys].replace('Mentioned', 1).replace('Not mentioned', 0).fillna(0).astype(int)
    #df_l = df_l[df_l.sum(axis=1) == 1]
    new_keys = [f'{key}_{l}' for l in ls]
    df_l = df_l.rename(dict(zip(keys, new_keys)), axis=1)
    df_l = df_l.reset_index().rename({'index': 'id'}, axis=1)
    long = pd.wide_to_long(df_l, stubnames=[key], i='id',j='lang', sep='_', suffix='.*')
    long = long[long[key] != 0]
    long = long.sort_values(by='id').reset_index().drop(key, axis=1).rename({'lang': key}, axis=1)
    
    return df_l

def rename(df, frm, to, ls, key):
    to = to + 1
    keys = [f'v{i}' for i in range(frm, to)]
    df_l = df[keys].replace('Mentioned', 1).replace('Not mentioned', 0).fillna(0).astype(int)
    #df_l = df_l[df_l.sum(axis=1) == 1]
    new_keys = [f'{key}_{l}' for l in ls]
    df_l = df_l.rename(dict(zip(keys, new_keys)), axis=1)
    return df_l


def open_survey(survey):
    path = f'../../../{survey}.sav'
    df = pd.read_spss(path)
    return df

def piv(df, nq, prefix="L2"):
    key = f'v{nq}'
    d = prefix + '_' + df[key].astype(str).to_frame().replace({'None': 'DK'}).fillna('DK')
    d['cnt'] = 1
    d = d.pivot(columns=key).fillna(0).astype(int)
    d.columns = d.columns.droplevel()
    return d

def concat_and_save(nat, L1, L2, survey):
    res = pd.concat([nat, L1, L2], axis=1)#.replace({None:  np.NaN, 'None': np.NaN})
    res = res.drop('L2_nan', axis=1, errors='ignore')
    res['Year'] = year
    res.to_csv(f'res/{survey}.csv')
    return res

In [3]:
survey = 'EB634_2005_ZA4411_v1-1-0'
year = 2005
df = open_survey(survey)
df.head()

,v1,v2,v3,version,v4,v5,v6,v7,v8,v9,...,v667,v668,v669,v670,v671,v672,v673,v674,v675,v676
0,4411.0,4411.0,1st za edition as of 30-11-2007,1.1.0 (2012-03-30),Eurobarometer 63.4,1.0,Belgium,BE,1.0739,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V414),1000001.0,105012486.0
1,4411.0,4411.0,1st za edition as of 30-11-2007,1.1.0 (2012-03-30),Eurobarometer 63.4,2.0,Belgium,BE,0.8147,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Students (2 in V414),1000002.0,105012486.0
2,4411.0,4411.0,1st za edition as of 30-11-2007,1.1.0 (2012-03-30),Eurobarometer 63.4,3.0,Belgium,BE,0.8771,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Self-employed (5 to 9 in V414),1000003.0,105012486.0
3,4411.0,4411.0,1st za edition as of 30-11-2007,1.1.0 (2012-03-30),Eurobarometer 63.4,4.0,Belgium,BE,1.0070,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Self-employed (5 to 9 in V414),1000004.0,105012486.0
4,4411.0,4411.0,1st za edition as of 30-11-2007,1.1.0 (2012-03-30),Eurobarometer 63.4,5.0,Belgium,BE,0.6132,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Self-employed (5 to 9 in V414),1000005.0,105012486.0


In [5]:
with open('../../../languages_634.txt') as fp:
    ls = fp.read().split('\n')

L1 = rename(df, 442, 476, ls, "L1")
L2 = rename(df, 480, 514, ls, "L2")

for nq in range(477, 480):
    d = piv(df, nq)
    L2 = L2 + d
    
nat = df[['v6', 'v7']].rename({'v6': 'Nationality', 'v7': 'ISO_Nationality'}, axis=1)
res = concat_and_save(nat, L1, L2, survey)
res.groupby('Nationality').sum()

,L1_German,L1_English,L1_Arabic,L1_Bulgarian,L1_Catalan,L1_Chinese,L1_Croatian,L1_Danish,L1_Spanish,L1_Estonian,...,L2_Portuguese,L2_Romanian,L2_Russian,L2_Sign language,L2_Slovak,L2_Slovenian,L2_Spanish,L2_Swedish,L2_Turkish,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Austria,966,10,0,2,0,0,7,0,0,0,...,1,4,12,18,4,14,32,3,2,2005000
Belgium,2,1,15,1,0,2,1,0,2,0,...,7,1,4,2,0,0,39,0,1,2005000
Bulgaria,0,1,1,893,0,0,0,0,0,1,...,2,13,212,0,0,0,10,0,29,2041090
Croatia,0,2,0,0,0,0,979,0,0,0,...,2,1,41,3,7,71,18,0,1,2005000
Cyprus (Republic),2,3,1,0,0,0,0,0,0,0,...,0,2,5,0,0,0,8,0,6,1012525
Cyprus (TCC),0,8,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,1,1002500
Czech Republic,3,1,0,3,0,1,0,0,1,0,...,0,0,205,1,172,1,6,0,0,2171415
Denmark,4,6,1,0,0,0,0,1019,0,0,...,3,0,5,1,0,0,33,200,1,2107255
Estonia,0,2,0,0,0,0,0,0,0,812,...,0,0,621,0,0,0,3,3,0,2007005


In [6]:
survey = 'EB643_2005_ZA4415_v1-0-1'
year = 2005
df = open_survey(survey)
df.head()

,v1,v2,v3,version,v4,v5,v6,v7,v8,v9,...,v1163,v1164,v1165,v1166,v1167,v1168,v1169,v1170,v1171,v1172
0,4415.0,4415.0,"1st GESIS-ZA edition as of September 5, 2008",1.0.1 (2012-03-30),Eurobarometer 64.3,1.0,Belgium,BE,0.7143,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V1062),Questionnaire Version B,1000001.0,105045868.0
1,4415.0,4415.0,"1st GESIS-ZA edition as of September 5, 2008",1.0.1 (2012-03-30),Eurobarometer 64.3,2.0,Belgium,BE,1.0302,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V1062),Questionnaire Version A,1000002.0,105045868.0
2,4415.0,4415.0,"1st GESIS-ZA edition as of September 5, 2008",1.0.1 (2012-03-30),Eurobarometer 64.3,3.0,Belgium,BE,0.6693,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V1062),Questionnaire Version A,1000003.0,105045868.0
3,4415.0,4415.0,"1st GESIS-ZA edition as of September 5, 2008",1.0.1 (2012-03-30),Eurobarometer 64.3,4.0,Belgium,BE,0.6693,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V1062),Questionnaire Version B,1000004.0,105045868.0
4,4415.0,4415.0,"1st GESIS-ZA edition as of September 5, 2008",1.0.1 (2012-03-30),Eurobarometer 64.3,5.0,Belgium,BE,0.7190,Not United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,Manual workers (15 to 18 in V1062),Questionnaire Version B,1000005.0,105045868.0


In [7]:
nat = df[['v6', 'v7']].rename({'v6': 'Nationality', 'v7': 'ISO_Nationality'}, axis=1)
nat.head()

,Nationality,ISO_Nationality
0,Belgium,BE
1,Belgium,BE
2,Belgium,BE
3,Belgium,BE
4,Belgium,BE


In [8]:
with open('../../../languages_643.txt') as fp:
    ls = fp.read().split('\n')
list(zip(ls, range(1, len(ls) +1)))

[('German', 1),
 ('English', 2),
 ('Arabic', 3),
 ('Bulgarian', 4),
 ('Catalan', 5),
 ('Chinese', 6),
 ('Croatian', 7),
 ('Danish', 8),
 ('Spanish', 9),
 ('Estonian', 10),
 ('Finnish', 11),
 ('French', 12),
 ('Greek', 13),
 ('Hungarian', 14),
 ('Italian', 15),
 ('Latvian', 16),
 ('Lithuanian', 17),
 ('Luxembourgish', 18),
 ('Maltese', 19),
 ('Dutch', 20),
 ('Polish', 21),
 ('Portuguese', 22),
 ('Romanian', 23),
 ('Russian', 24),
 ('Slovak', 25),
 ('Slovenian', 26),
 ('Swedish', 27),
 ('Czech', 28),
 ('Turkish', 29),
 ('Irish/Gaelic', 30),
 ('Basque', 31),
 ('Galician', 32),
 ('Other regional language (e.g. Welsh, Cornish)', 33),
 ('Hindi', 34),
 ('Urdu', 35),
 ('Punjabi', 36),
 ('Gujarati', 37),
 ('Bengali', 38),
 ('Other', 39),
 ('DK', 40)]

In [9]:
L1 = rename(df, 264, 303, ls, "L1")
L1

,L1_German,L1_English,L1_Arabic,L1_Bulgarian,L1_Catalan,L1_Chinese,L1_Croatian,L1_Danish,L1_Spanish,L1_Estonian,...,L1_Basque,L1_Galician,"L1_Other regional language (e.g. Welsh, Cornish)",L1_Hindi,L1_Urdu,L1_Punjabi,L1_Gujarati,L1_Bengali,L1_Other,L1_DK
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
L2 = pd.DataFrame(index=L1.index, columns=[f"L2_{l}" for l in ls]).fillna(0)

for nq in range(304, 307):
    d = piv(df, nq)
    L2 = L2 + d

res = concat_and_save(nat, L1, L2, survey)  
res.groupby('Nationality').sum()

,L1_German,L1_English,L1_Arabic,L1_Bulgarian,L1_Catalan,L1_Chinese,L1_Croatian,L1_Danish,L1_Spanish,L1_Estonian,...,L2_Punjabi,L2_Romanian,L2_Russian,L2_Slovak,L2_Slovenian,L2_Spanish,L2_Swedish,L2_Turkish,L2_Urdu,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Austria,959,3,4,0,1,1,9,0,0,0,...,0,1,16,0,13,37,2,3,0,2009010
Belgium,4,3,13,0,0,0,0,0,5,0,...,0,2,2,0,0,54,0,0,0,2005000
Bulgaria,1,1,0,902,0,0,0,0,0,0,...,0,8,350,0,0,18,1,25,0,2013020
Croatia,0,0,0,0,0,0,982,0,0,0,...,0,1,33,6,50,21,1,0,0,2005000
Cyprus (Republic),0,5,0,0,0,0,0,0,0,0,...,0,3,10,0,0,7,2,8,0,1006510
Cyprus (TCC),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1002500
Czech Republic,3,1,0,2,0,0,1,0,0,0,...,0,1,221,163,2,2,1,0,0,2063145
Denmark,4,3,3,0,1,0,1,998,3,1,...,0,0,7,0,0,50,120,2,0,2067155
Estonia,3,0,0,0,0,1,0,0,0,824,...,0,0,662,0,0,1,5,0,0,2005000


In [11]:
survey = 'EB551_2001_ZA3507_v1-0-1'
year = 2001
df = open_survey(survey)
df.head()

,v1,v2,v3,version,v4,v5,isocntry,v6,v7,v8,...,v438,v439,v440,v441,v442,v443,v444,v445,v446,v447
0,3507.0,3507.0,"2nd za edition (November 21, 2006)",1.0.1 (2012-03-30),Eurobarometer 55.1,1.0,BE,Belgium,1.0119,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),"+ (coded 3, 4, 5 or 0 for V70 and 1 for V69; o...",Skilled manual workers,"+++ (coded 1 or 2 for V91, V92 and V93)",Manual workers (coded 15 to 18 in V362),Euro countries,Euro countries,10001.0
1,3507.0,3507.0,"2nd za edition (November 21, 2006)",1.0.1 (2012-03-30),Eurobarometer 55.1,2.0,BE,Belgium,0.9588,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),"- (coded 3, 4, 5 or 0 for V70 and 2 for V69; ...",Other (unskilled) manual worker,NaN,Unemployed (coded 3 in V362),Euro countries,Euro countries,10002.0
2,3507.0,3507.0,"2nd za edition (November 21, 2006)",1.0.1 (2012-03-30),Eurobarometer 55.1,3.0,BE,Belgium,0.9586,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),"- - low (coded 3, 4, 5 or 0 for V70 and 3 for ...",NaN,"+++ (coded 1 or 2 for V91, V92 and V93)",Manual workers (coded 15 to 18 in V362),Euro countries,Euro countries,10003.0
3,3507.0,3507.0,"2nd za edition (November 21, 2006)",1.0.1 (2012-03-30),Eurobarometer 55.1,4.0,BE,Belgium,0.9586,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),"- (coded 3, 4, 5 or 0 for V70 and 2 for V69; ...","Employed position, not at a desk but in a serv...",NaN,Manual workers (coded 15 to 18 in V362),Euro countries,Euro countries,10004.0
4,3507.0,3507.0,"2nd za edition (November 21, 2006)",1.0.1 (2012-03-30),Eurobarometer 55.1,5.0,BE,Belgium,1.0107,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),"- (coded 3, 4, 5 or 0 for V70 and 2 for V69; ...","Employed position, working mainly at a desk","+++ (coded 1 or 2 for V91, V92 and V93)",Retired (coded 4 in V362),Euro countries,Euro countries,10005.0


In [12]:
nat = df[['v6', 'isocntry']].rename({'v6': 'Nationality', 'isocntry': 'ISO_Nationality'}, axis=1)
nat.head()

,Nationality,ISO_Nationality
0,Belgium,BE
1,Belgium,BE
2,Belgium,BE
3,Belgium,BE
4,Belgium,BE


In [13]:
with open('../../../languages_551.txt') as fp:
    ls = fp.read().split('\n')
L2 = rename(df, 39, 53, ls, 'L2')
L2

,L2_Danish,L2_German,L2_French,L2_Italian,L2_Dutch,L2_English,L2_Spanish,L2_Portuguese,L2_Greek,L2_Swedish,L2_Finnish,L2_Arabic,L2_Chinese,L2_Other,L2_DK
0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16158,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16159,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16161,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
L1 = piv(df, 38, "L1")
L1

v38,L1_Arabic,L1_Chinese,L1_Danish,L1_Dutch,L1_English,L1_Finnish,L1_French,L1_German,L1_Greek,L1_Italian,L1_Other,L1_Portuguese,L1_Spanish,L1_Swedish
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16158,0,0,0,0,0,0,0,1,0,0,0,0,0,0
16159,0,0,0,0,0,0,0,1,0,0,0,0,0,0
16160,0,0,0,0,0,0,0,1,0,0,0,0,0,0
16161,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [15]:
res = concat_and_save(nat, L1, L2, survey)  
res.groupby('Nationality').sum()

,L1_Arabic,L1_Chinese,L1_Danish,L1_Dutch,L1_English,L1_Finnish,L1_French,L1_German,L1_Greek,L1_Italian,...,L2_Spanish,L2_Portuguese,L2_Greek,L2_Swedish,L2_Finnish,L2_Arabic,L2_Chinese,L2_Other,L2_DK,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Austria,1,0,2,1,3,0,3,998,1,0,...,27,8,7,2,1,1,1,26,378,2071035
Belgium,4,0,0,539,5,0,433,7,2,24,...,42,3,2,1,1,4,0,25,403,2095047
Denmark,1,0,981,0,3,0,0,4,0,0,...,28,3,1,85,2,0,1,38,176,2005002
Finland,0,0,0,0,3,945,0,1,0,0,...,14,1,2,306,58,0,1,27,454,2045022
France,8,2,0,2,6,0,952,3,0,9,...,102,7,1,1,0,15,1,35,513,2009004
Germany (East),0,0,6,0,1,0,1,1040,1,2,...,6,2,1,1,0,1,0,123,652,2103051
Germany (West),0,0,9,1,2,0,3,992,0,22,...,32,3,5,1,1,1,0,62,442,2071035
Greece,0,0,0,0,1,0,0,1,989,0,...,7,1,15,0,0,4,0,41,566,2011005
Ireland,0,0,0,2,900,0,1,1,0,0,...,27,2,2,0,0,1,0,105,655,1940970


In [27]:
survey = 'EB520_1999_ZA3204_v1-0-1'
year = 1999
df = open_survey(survey)
df = df.replace({'Not Mentioned': 0})
df.head()

,v1,v2,v3,version,v4,v5,isocntry,v6,v7,v8,...,v884,v885,v886,v887,v888,v889,v890,v891,v892,v893
0,3204.0,3204.0,2nd za edition - release 04-06-2007,1.0.1 (2012-03-30),Eurobarometer 52.0,1.0,BE,Belgium,1.2224,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),+,NaN,(V) The Green Group in the European Parliament,++,Students (coded 2 in V809),Euro countries,12274.0
1,3204.0,3204.0,2nd za edition - release 04-06-2007,1.0.1 (2012-03-30),Eurobarometer 52.0,2.0,BE,Belgium,0.8667,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),- - low,Skilled manual workers,NaN,+++,Manual workers (coded 15 to 18 in V809),Euro countries,12150.0
2,3204.0,3204.0,2nd za edition - release 04-06-2007,1.0.1 (2012-03-30),Eurobarometer 52.0,3.0,BE,Belgium,0.7328,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),-,"Employed position, working mainly at a desk",NaN,++,Manual workers (coded 15 to 18 in V809),Euro countries,11531.0
3,3204.0,3204.0,2nd za edition - release 04-06-2007,1.0.1 (2012-03-30),Eurobarometer 52.0,4.0,BE,Belgium,1.1460,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),++ high,Middle management,(PSE) Group of the Party of European Socialists,+++,Managers (coded 10 to 12 in V809),Euro countries,11494.0
4,3204.0,3204.0,2nd za edition - release 04-06-2007,1.0.1 (2012-03-30),Eurobarometer 52.0,5.0,BE,Belgium,0.9855,Belgium,...,NaN,NaN,(COUNTRY SPECIFIC),+,Skilled manual workers,(N.I.) Non-attached,--,House persons (coded 1 in V809),Euro countries,11379.0


In [28]:
nat = df[['v6', 'isocntry']].rename({'v6': 'Nationality', 'isocntry': 'ISO_Nationality'}, axis=1)
nat.head()

,Nationality,ISO_Nationality
0,Belgium,BE
1,Belgium,BE
2,Belgium,BE
3,Belgium,BE
4,Belgium,BE


In [29]:
with open('../../../languages_520.txt') as fp:
    ls = fp.read().split('\n')

L2 = rename(df, 49, 63, ls, 'L2')
L2

,L2_NA,L2_Danish,L2_German,L2_French,L2_Italian,L2_Dutch,L2_English,L2_Spanish,L2_Portuguese,L2_Greek,L2_Swedish,L2_Finnish,L2_Arabic,L2_Chinese,L2_Other
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16066,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16067,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16068,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16069,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [30]:
L1 = piv(df, 38, "L1")
L1

v38,L1_Arabic,L1_Chinese,L1_Danish,L1_Dutch,L1_English,L1_Finnish,L1_French,L1_German,L1_Greek,L1_Italian,L1_Other (SPECIFY),L1_Portuguese,L1_Spanish,L1_Swedish,L1_nan
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16066,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
16067,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
16068,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
16069,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [31]:
res = concat_and_save(nat, L1, L2, survey)  
res.groupby('Nationality').sum()

,L1_Arabic,L1_Chinese,L1_Danish,L1_Dutch,L1_English,L1_Finnish,L1_French,L1_German,L1_Greek,L1_Italian,...,L2_English,L2_Spanish,L2_Portuguese,L2_Greek,L2_Swedish,L2_Finnish,L2_Arabic,L2_Chinese,L2_Other,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Austria,1,1,5,2,5,0,0,985,1,0,...,18,3,3,2,0,1,0,9,417,2034982
Belgium,1,0,3,551,5,0,423,4,3,34,...,39,3,2,2,0,5,0,27,371,2086956
Denmark,0,0,973,1,12,1,1,2,0,0,...,22,1,1,77,0,1,1,39,162,2000999
Finland,0,0,0,0,0,952,0,0,0,0,...,10,1,0,357,47,0,0,24,389,2028985
France,11,1,0,0,2,0,957,1,0,6,...,73,5,1,0,1,6,3,17,580,2004997
Germany (East),0,0,2,0,1,0,1,1007,0,0,...,4,0,0,0,0,1,0,115,675,2038980
Germany (West),0,0,2,0,3,0,4,984,4,3,...,23,2,3,2,2,3,0,58,472,2034982
Greece,0,0,1,0,0,0,1,0,1006,1,...,10,0,4,0,0,0,0,3,544,2018990
Ireland,0,0,0,5,926,0,4,0,0,2,...,15,0,0,0,0,1,2,85,711,2000999


In [54]:
survey = 'EB440_1995_ZA2689_v1-0-1'
year = 1995
df = open_survey(survey)
df = df.replace({'NOT MENTIONED': 0, 'MENTIONED': 1})
df.head()

,v1,v2,version,v3,v4,isocntry,v5,v6,v7,v8,...,v620,v621,v622,v623,v624,v625,v626,v627,v628,v629
0,2689.0,2ND CODEBOOK EDITION (release 28-08.2006),1.0.1 (2012-07-01),Eurobarometer 44.0,1.0,BE,BELGIUM,1.2000,BELGIUM,1.2192,...,<country specific>,YES,<COUNTRY SPECIFIC>,-,POSITIVE,MIDDLE MANAGEMENT,ELDR,B,SELF-EMPLOYED,20095.0
1,2689.0,2ND CODEBOOK EDITION (release 28-08.2006),1.0.1 (2012-07-01),Eurobarometer 44.0,2.0,BE,BELGIUM,0.7544,BELGIUM,0.7665,...,<country specific>,YES,<COUNTRY SPECIFIC>,-,AMBIVALENT,SKILLED MAN WORKERS,PSE,D,MANUAL WORKERS,20006.0
2,2689.0,2ND CODEBOOK EDITION (release 28-08.2006),1.0.1 (2012-07-01),Eurobarometer 44.0,3.0,BE,BELGIUM,1.2274,BELGIUM,1.2470,...,<country specific>,YES,<COUNTRY SPECIFIC>,- - LOW,POSITIVE,SUPERVISORS,NaN,D,NaN,20029.0
3,2689.0,2ND CODEBOOK EDITION (release 28-08.2006),1.0.1 (2012-07-01),Eurobarometer 44.0,4.0,BE,BELGIUM,0.8603,BELGIUM,0.8741,...,<country specific>,YES,<COUNTRY SPECIFIC>,- - LOW,POSITIVE,NaN,PSE,NaN,NaN,20007.0
4,2689.0,2ND CODEBOOK EDITION (release 28-08.2006),1.0.1 (2012-07-01),Eurobarometer 44.0,5.0,BE,BELGIUM,0.8458,BELGIUM,0.8593,...,<country specific>,YES,<COUNTRY SPECIFIC>,-,AMBIVALENT,SHOP OWNER/CRAFTSMEN,ELDR,C,MANUAL WORKERS,20010.0


In [60]:
nat = df[['v5', 'isocntry']].rename({'v5': 'Nationality', 'isocntry': 'ISO_Nationality'}, axis=1)
nat.Nationality = nat.Nationality.str.title()
nat.head()

,Nationality,ISO_Nationality
0,Belgium,BE
1,Belgium,BE
2,Belgium,BE
3,Belgium,BE
4,Belgium,BE


In [61]:
with open('../../../languages_440.txt') as fp:
    ls = fp.read().split('\n')

L1 = rename(df, 264, 280, ls, 'L1')
L1

,L1_Danish,L1_German,L1_French,L1_Italian,L1_Dutch,L1_English,L1_Spanish,L1_Portug,L1_Greek,L1_Swedish,L1_Finnish,L1_Russian,L1_Arabic,L1_Chinese,L1_Japanese,L1_Other,L1_DK
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16636,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16637,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16638,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16639,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [62]:
L2 = rename(df, 281, 297, ls, 'L2')
L2

,L2_Danish,L2_German,L2_French,L2_Italian,L2_Dutch,L2_English,L2_Spanish,L2_Portug,L2_Greek,L2_Swedish,L2_Finnish,L2_Russian,L2_Arabic,L2_Chinese,L2_Japanese,L2_Other,L2_DK
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16636,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
16637,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
16638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [63]:
res = concat_and_save(nat, L1, L2, survey)  
res.groupby('Nationality').sum()

,L1_Danish,L1_German,L1_French,L1_Italian,L1_Dutch,L1_English,L1_Spanish,L1_Portug,L1_Greek,L1_Swedish,...,L2_Greek,L2_Swedish,L2_Finnish,L2_Russian,L2_Arabic,L2_Chinese,L2_Japanese,L2_Other,L2_DK,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Austria,5,980,3,1,0,1,0,0,1,0,...,6,2,2,6,3,0,1,30,434,2004975
Belgium,1,6,394,27,546,4,9,1,1,0,...,3,0,2,2,6,1,2,20,350,2026920
Denmark,982,2,0,0,1,3,2,0,1,1,...,3,213,3,5,5,1,0,36,156,1995000
East Germany,3,1081,5,0,0,1,1,0,0,0,...,1,0,0,178,0,0,1,19,736,2182530
Finland,3,4,0,0,0,1,0,0,1,68,...,2,357,56,18,0,0,2,12,419,2058840
France,0,7,959,8,1,1,13,4,0,1,...,2,2,1,3,7,1,2,14,495,2008965
Greece,0,5,1,0,0,1,2,0,997,0,...,9,2,1,10,5,0,1,43,549,2006970
Ireland,0,3,2,2,4,945,3,0,0,1,...,3,1,1,3,3,1,2,103,677,2004975
Italy,0,2,1,1015,2,2,2,0,0,0,...,1,0,0,1,0,0,1,12,553,2042880


In [65]:
survey = 'EB340_1990_ZA1960_v1-0-1'
year = 1990
df = open_survey(survey)
df = df.replace({'NOT MENTIONED': 0, 'MENTIONED': 1})
df.head()

,v1,v2,v3,version,v4,v5,isocntry,v6,v7,v8,...,v533,v534,v535,v536,v537,v538,v539,v540,v541,v542
0,ICPSR STUDY NUMBER,ZA STUDY NUMBER,2ND ICPSR EDITION,1.0.1 (2012-07-01),EUROBAROMETER 34.0,5446.0,FR,FRANCE,0.5275,FRANCE,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,ICPSR STUDY NUMBER,ZA STUDY NUMBER,2ND ICPSR EDITION,1.0.1 (2012-07-01),EUROBAROMETER 34.0,5447.0,FR,FRANCE,0.9127,FRANCE,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,NaN
2,ICPSR STUDY NUMBER,ZA STUDY NUMBER,2ND ICPSR EDITION,1.0.1 (2012-07-01),EUROBAROMETER 34.0,5448.0,FR,FRANCE,0.7643,FRANCE,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
3,ICPSR STUDY NUMBER,ZA STUDY NUMBER,2ND ICPSR EDITION,1.0.1 (2012-07-01),EUROBAROMETER 34.0,5449.0,FR,FRANCE,1.0027,FRANCE,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,NaN
4,ICPSR STUDY NUMBER,ZA STUDY NUMBER,2ND ICPSR EDITION,1.0.1 (2012-07-01),EUROBAROMETER 34.0,5450.0,FR,FRANCE,0.5796,FRANCE,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN


In [68]:
nat = df[['v6', 'isocntry']].rename({'v6': 'Nationality', 'isocntry': 'ISO_Nationality'}, axis=1)
nat.Nationality = nat.Nationality.str.title()
nat.head()

,Nationality,ISO_Nationality
0,France,FR
1,France,FR
2,France,FR
3,France,FR
4,France,FR


In [72]:
with open('../../../languages_340.txt') as fp:
    ls = fp.read().split('\n')

L2 = rename(df, 194, 205, ls, 'L2')
L2

,L2_Danish,L2_German,L2_French,L2_Italian,L2_Dutch,L2_English,L2_Spanish,L2_Portuguese,L2_Greek,L2_Other,L2_None,L2_DK
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13878,0,0,0,0,0,1,0,0,0,0,0,0
13879,0,0,0,0,0,0,0,0,0,0,1,0
13880,0,0,0,0,0,0,0,0,0,0,0,0
13881,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
L1 = rename(df, 206, 217, ls, 'L1')
L1

,L1_Danish,L1_German,L1_French,L1_Italian,L1_Dutch,L1_English,L1_Spanish,L1_Portuguese,L1_Greek,L1_Other,L1_None,L1_DK
0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13878,0,0,0,0,0,0,0,0,0,0,0,0
13879,0,0,0,0,0,0,0,0,0,0,0,0
13880,0,0,0,0,0,0,0,0,0,0,0,0
13881,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
res = concat_and_save(nat, L1, L2, survey)  
res.groupby('Nationality').sum()

,L1_Danish,L1_German,L1_French,L1_Italian,L1_Dutch,L1_English,L1_Spanish,L1_Portuguese,L1_Greek,L1_Other,...,L2_Italian,L2_Dutch,L2_English,L2_Spanish,L2_Portuguese,L2_Greek,L2_Other,L2_None,L2_DK,Year
Nationality,,,,,,,,,,,,,,,,,,,,,
Belgium,0,2,435,2,565,13,1,0,0,9,...,24,119,304,26,2,1,11,434,2,1982040
Denmark,992,11,3,1,1,22,0,0,1,5,...,9,6,686,17,1,1,15,248,5,1990000
East Germany,3,1021,2,0,0,6,0,0,0,14,...,1,2,148,2,0,0,93,812,0,2031790
France,1,11,980,21,0,29,17,4,1,7,...,59,2,300,120,9,3,34,522,40,2033780
Greece,0,6,2,1,0,12,0,0,979,8,...,25,1,253,5,0,2,28,656,0,2005920
Ireland,1,2,7,1,1,1005,0,0,0,30,...,3,0,7,15,0,2,68,646,144,2033780
Italy,0,2,11,1051,0,11,4,0,0,11,...,18,1,203,23,3,1,2,705,12,2135270
Luxembourg,0,4,13,11,0,3,2,1,0,269,...,34,9,126,14,1,0,0,20,1,598990
Netherlands,0,15,4,2,1027,21,0,0,0,34,...,10,39,657,16,2,2,14,154,75,2123330
